In [1]:
# imports, run code in KPI_venv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
#import ibis
from datetime import datetime
import ast
from sklearn.datasets import load_digits
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from time import time
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import plotly.express as px
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_score
from sklearn.model_selection import GridSearchCV
import os
import random
import re
import string
#import nltk
import numpy as np
import pandas as pd
#from gensim.models import Word2Vec
#from nltk import word_tokenize
#from nltk.corpus import stopwords
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score
#import shap

In [3]:
import numpy as np
import pandas as pd
df = pd.read_csv('../data/Training_Data_2305.csv')
pd.set_option('display.max_colwidth', None)

In [ ]:
Test_Data = pd.read_excel('/home/jupyter/Test_data_March.xlsx')

In [4]:
import vertexai
from vertexai.language_models import TextGenerationModel
import multiprocessing
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing
import threading
model = TextGenerationModel.from_pretrained("text-bison@002")

vertexai.init(project="bt-netdetsds-aif-exp-prod", location="europe-west2")
parameters = {
     "candidate_count": 1,
    "max_output_tokens": 200,
    "temperature": 0,
    "top_p": 1 
}

"""If the text is less than 15 words then classify the text as "Not enough data to classify"
        If the text covers more than one category then classify the text with multiple different categories, include and between each topic, the max number of topics it should include should be ONLY 3.
        And only classify using the categories provided above and DO NOT suggest your own categories
        DO NOT LEAVE ANY output empty"""

2024-07-29 14:52:01.760055: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2024-07-29 14:52:01.760140: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


'If the text is less than 15 words then classify the text as "Not enough data to classify"\n        If the text covers more than one category then classify the text with multiple different categories, include and between each topic, the max number of topics it should include should be ONLY 3.\n        And only classify using the categories provided above and DO NOT suggest your own categories\n        DO NOT LEAVE ANY output empty'

In [5]:
%%time
categories = ['Account', 'Financial Operation', 'Broadband', 'Engineering', 'TV', 'Not Enough data to classify','Order', 'Hybrid Connect', 'EE Mobile', 'Digital Voice/Handset', 'Equipment', 'Disc', 'Broadband including Financial Operation', 'Broadband including Account', 'Account including Financial Operation']
#categories = ['broadband', 'engineering', 'account']


prompt = """
        CLASSIFY the text within the square brackets to be ONE of the following categories present within the following brackets and do NOT suggest your own category (
        
        "Account", "Financial Operation", "Broadband", "Engineering", "TV", "Not Enough data to classify","Order", "Hybrid Connect", "EE Mobile",
        "Digital Voice/Handset", "Equipment", "Disc", "Broadband including Financial Operation", "Broadband including Account", "Account including Financial Operation"
        
        )
        
        You need to classify the given text meaningfully & accurately into ONE of the following categories based on the context of the notes & not based on the keywords: 

         1. "Order" if the text contains words like orders, order, open da delivery and order status.
         2. "Account" if the text contains words like account, ee account, password, email, ee id, eeid, login issue, address issue, email issue. Just having the word 'account' need not necessarily mean account category as it needs to be based on the context & meaning of the issue.
         3. "Financial operation" if the text contains words like bill,  waive off, refund, payment, overpayment, charged, debit, credit, package, money, compensation, finance.
         4. "Digital Voice/Handset" if the text contains words like voice, unable to make or receive calls, no voice service, digital voice, dv phone, dv handset, dv service, no dial tone. 
         5. "Broadband" if the text contains words like  broadband, broadband fault, broadband connection, slow speed, speedtest
         6. "TV" if the text contains words like TV, set top box, xbox, sky, channels, and tv.         
         7. "Not enough data" if text are ambiguous & none of the above categories are applicable.
         8. "Equipment" if the text contains the words like returning equipment, ruturn or equipment.
         9. "Hybrid Connect" if the texts contains the word hybrid connect.
         10. "Disc" if the text contains words like wifi discs, discs, wifi disc or disc.
         11. "Broadband including Financial Operation" if the text contains words billing OR payment AND broadband.
         12. "Broadband including Account" if the text contains words brodband AND account. 
         13. "Account including Financial Operation" if the text contains words account OR email AND billing.
         14. "EE Mobile" if the text contains the word "EE Mobile"
         15. "Engineering" if the text contains words like booked engineer, qube for, engineering and appointment.
         
        
        <example>
        Text: wants her original number back but its bt and linked to another account which is still active passed cx to loyalty to get the bt acc ceased with no httc
        The answer is: Account
        
        Text: cx called in as they recently came over to fibre broadband engineer came out but couldnt do the job at that time and was rescheduled for but he was unaware of the times weve disconnected him before he could get visit int notes say that we needed to get an engineer with flat roof skills and could get the next update on the tried reconnecting but no restricted products call dropped after id told him wed have an update on the
        The answer is: Engineering
        
        Text: reason for contact received letter to set up direct debit but cx already set it up action taken dd set up for the broadband and mobile next action nfar
        The answer is: Broadband including Financial Operation
        
        Text: highlow high reason for cx called fro slow internet speed and dropped in broadband connection resolution warm transfered to ee tech team online acc promoted billing payment date anything else
        The answer is: Broadband including Financial Operation
        
        Text: transfer to tech to look into potential broadband fault sent off application request form to assist with merge old and new ee accounts
        The answer is: Broadband including Account
        
        Text: cx moved too ee broadband old account was with bt in fathers name has no broadband open ordr unable to warm transfer as issue with cce
        The answer is: Broadband including Account
        
        Text: called in to explain he cant activate his ee id when he types his email address in this due to email spelt wrong edited the email on account 
        The answer is: Account
        
        Text: order has now been submitted customer asked to wait 4 days
        The answer is: Order
        
        Text: asked about appointment and when the engineer with arrive
        The answer is: Engineering
        
        Text: engineer has been booked and appointment date is set 
        The answer is: Engineering
        
        Text: hybrid connect not working properly and have been asked to return the product
        The answer is: Hybrid Connect
        
        Text: reason for contact want to return equipment cx not recieved returns returns bag after checking account cx move from bt to new ee cx needs returns bag action taken successfuly replaced returns bag
        The answer is: Equipment
        
        Text: mrs recieved email about returning equipment not sure what this is about regraded package in store smart hub sent out advised the return email is about her smart hub she currently has
        The answer is: Equipment
        
        Text: rang to chase up manager call back have emailed both manager and advisor to call cx back as its overdue can see of the actions was to get hybrid connect taken off
        The answer is: Hybrid Connect
        
        Text: dv issues with phone, customer cannot hear anything when calling 
        The answer is: Digital Voice/Handset
        
        Text: daughter called to see if there was any issues with her mums order checked and comp she adv phone service was not working adv her to make sure conn to back of the hub to green post as not dv handset
        The answer is: Digital Voice/Handset
        
        Text: cst moved to new ee and wifi discs dont work cst still needs as has blackspots explained would need smart wifi and called value
        The answer is: Disc
        
        Text:has bt wifi discs advised they are not compatable with ee hub plus ref cust to online shop on advice from ee value team as cust would like to purchase outright
        The answer is: Disc
        
        Text: called to know why bill was high and why promotion wasn't added
        The answer is: Financial Operation
        
        Text: transferred in hes chasing his order an engineer called to fit the broadband but more work is required have transferred him to new ee connections
        The answer is: Broadband 
        
        Text: cx said that his bt wifi disc is not compatible with hub had look into his package and he is not on the complete wifi package was in queue to new ee value for so had to transfer for them to put cx on complete wifi package for him to have new ee discs
        The answer is: Disc
        
        Text: customer having issues with connecting to discs and need helps with set up a
        The answer is: Disc
        
        Text: cst called adv has switched over to ee from has has been taken off spoke to ee mobile and warm trans call
        The answer is: EE Mobile
        
        Text: customer has changed from bt to new trying to link broadband to his eeid which he is also trying to link to his mobile account also has not received an ee sim he ordered transferred to ee mobile
        The answer is: EE Mobile
        
        Text: ah working broadband connecion wnat inactuve
        The answer is: Broadband
        
        Text: broadband connecion not working
        The answer is: Broadband 
        
        Text: broadband connections keeps dropping and issues with connecting to hub
        The answer is: Broadband 
        
        Text: was adding onto her now package and was telling her to call us and give us code have leased and transferred to tv tech for further assistance
        The answer is: TV
        
        Text: customer called regarding issues with sky sport missing channels this is now customer error code broadband is connected bt preview is working home checks done tv box reboot software update factory reset and that fixed it
        The answer is: TV
        </example>
        
    
        And ONLY CLASSIFY into ONE category using the categories provided above and DO NOT suggest your own categories
        
        Finally, DO NOT give explaination, Return ONLY the category name with no additional characters.
        
        """

def find_topic(model, parameters, prompt, text):
    response = model.predict(f"{prompt}\n[{text}]\n", **parameters)
    if response.text.replace('**', '').replace('""', '').strip() in categories:
        return response.text.replace('**', '').replace('""', '').strip()
    else:
        return ('Unknown')

def apply_find_topic_parallel(comments, prompt, model, parameters):
    num_cpus = multiprocessing.cpu_count()
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(find_topic, model, parameters, prompt, comment) for comment in comments]
        results = []
        for future in as_completed(futures):
            results.append(future.result())
        return results

comments = df["clean"].to_list()
topics = apply_find_topic_parallel(comments, prompt, model, parameters)
df['Topic_TB'] = topics

#Validate equipment predictions
#Validate New EE prediction
#relook at temp
#

#Add examples
#equipment 
#
"""def find_topic(model, parameters, prompt, text):
    response = model.predict(f"{prompt}\n[{text}]\n", **parameters)
    if response.text.strip() in categories:
        return response.text.strip()
    else:
        return ('Unknown')"""


CPU times: user 1min 26s, sys: 1min 8s, total: 2min 35s
Wall time: 13min 53s


'def find_topic(model, parameters, prompt, text):\n    response = model.predict(f"{prompt}\n[{text}]\n", **parameters)\n    if response.text.strip() in categories:\n        return response.text.strip()\n    else:\n        return (\'Unknown\')'

In [6]:
df['totalwords'] = df['clean'].str.count(' ') + 1

def selector(df):
    if (df['totalwords'] in range(0,16) ) :
        return ('Y')
    elif (df['totalwords'] >= 16) :
        return ('N')
    else:
        return False

df['Short Text'] = df.apply(
    lambda df : selector(df), axis=1
    )
df

,COMMENTS_LONG,tokens,clean,Without PII,Topic,Category,ASSET_NUM,Words,Second Topic,Second Category,Vlookup,Topic_TB,totalwords,Short Text
0,"\naccount holder\n\ncmr has called as she is having issues with her dv service - unable to make or receive calls \n\nwhen making a call - she receives the following error message ""no voice service on the hub""\n\ndv service showing as red via hv\n\nactioned the following \n- regenerate sip\n\nissue now resolved","['account', 'holder', 'cmr', 'has', 'called', 'as', 'she', 'is', 'having', 'issues', 'with', 'her', 'dv', 'service', 'unable', 'to', 'make', 'or', 'receive', 'calls', 'when', 'making', 'call', 'she', 'receives', 'the', 'following', 'error', 'message', 'no', 'voice', 'service', 'on', 'the', 'hub', 'dv', 'service', 'showing', 'as', 'red', 'via', 'hv', 'actioned', 'the', 'following', 'regenerate', 'sip', 'issue', 'now', 'resolved']",account holder cmr has called as she is having issues with her dv service unable to make or receive calls when making call she receives the following error message no voice service on the hub dv service showing as red via hv actioned the following regenerate sip issue now resolved,account holder cmr has called as she is having issues with her dv service unable to make or receive calls when making call she receives the following error message no voice service on the hub dv service showing as red via hv actioned the following regenerate sip issue now resolved,3,Digital Voice,NaN,27.0,0.0,Call-related problems,Call-related problems,Digital Voice/Handset,50,N
1,issue with home phone - was reset and same issue\ncan receive calls but cant make calls as engaged tone\nvoicemail wont load either to check if any vm messages causing issue\nsip regen done and rebooted hub\ncall dropped to mobile eventually\ncalled cmr back twice but no answer\nleft vm to advise to try again and call us back if further checks needed\nsystem advises to send to cpu if still having issues,"['issue', 'with', 'home', 'phone', 'was', 'reset', 'and', 'same', 'issue', 'can', 'receive', 'calls', 'but', 'cant', 'make', 'calls', 'as', 'engaged', 'tone', 'voicemail', 'wont', 'load', 'either', 'to', 'check', 'if', 'any', 'vm', 'messages', 'causing', 'issue', 'sip', 'regen', 'done', 'and', 'rebooted', 'hub', 'call', 'dropped', 'to', 'mobile', 'eventually', 'called', 'cmr', 'back', 'twice', 'but', 'no', 'answer', 'left', 'vm', 'to', 'advise', 'to', 'try', 'again', 'and', 'call', 'us', 'back', 'if', 'further', 'checks', 'needed', 'system', 'advises', 'to', 'send', 'to', 'cpu', 'if', 'still', 'having', 'issues']",issue with home phone was reset and same issue can receive calls but cant make calls as engaged tone voicemail wont load either to check if any vm messages causing issue sip regen done and rebooted hub call dropped to mobile eventually called cmr back twice but no answer left vm to advise to try again and call us back if further checks needed system advises to send to cpu if still having issues,issue with home phone was reset and same issue can receive calls but cant make calls as engaged tone voicemail wont load either to check if any vm messages causing issue sip regen done and rebooted hub call dropped to mobile eventually called cmr back twice but no answer left vm to advise to try again and call us back if further checks needed system advises to send to cpu if still having issues,3,Digital Voice,3-810536210337,50.0,0.0,Call-related problems,Call-related problems,Digital Voice/Handset,74,N
2,customer has called in to report that the dv handsets are not working \n\nno dial tone and cant make or receive any calls \n\nchecked hv\n\nif you phone the customer goes straight to vm\n\n\tvol055-9371457712472aims to be fixed by this has fixed the issue \n\nnfar,"['customer', 'has', 'called', 'in', 'to', 'report', 'that', 'the', 'dv', 'handsets', 'are', 'not', 'working', 'no', 'dial', 'tone', 'and', 'cant', 'make', 'or', 'receive', 'any', 'calls', 'checked', 'hv', 'if', 'you', 'phone', 'the', 'cu

In [7]:
short_text =  df[df['totalwords'] < 16 ]
short_text

,COMMENTS_LONG,tokens,clean,Without PII,Topic,Category,ASSET_NUM,Words,Second Topic,Second Category,Vlookup,Topic_TB,totalwords,Short Text
363,"get digital voice equipment\nno last caller anyone that rings, it cuts off \ncheckin","['get', 'digital', 'voice', 'equipment', 'no', 'last', 'caller', 'anyone', 'that', 'rings', 'it', 'cuts', 'off', 'checkin']",get digital voice equipment no last caller anyone that rings it cuts off checkin,get digital voice equipment no last caller anyone that rings it cuts off checkin,3,Digital Voice,NaN,26.0,3.0,Call cutting,Call cutting,Digital Voice/Handset,14,Y
556,"customer called as wanting voicemail removed off account.\n\nreceived error in agent, sent to manager.","['customer', 'called', 'as', 'wanting', 'voicemail', 'removed', 'off', 'account', 'received', 'error', 'in', 'agent', 'sent', 'to', 'manager']",customer called as wanting voicemail removed off account received error in agent sent to manager,customer called as wanting voicemail removed off account received error in agent sent to manager,0,Account,3-818403344552,11.0,NaN,NaN,NaN,Account,15,Y
583,"caller\naccount holder\nspoke with account holder, elisha\nhv ptm\nswitch query\nprovided information","['caller', 'account', 'holder', 'spoke', 'with', 'account', 'holder', 'elisha', 'hv', 'ptm', 'switch', 'query', 'provided', 'information']",caller account holder spoke with account holder elisha hv ptm switch query provided information,caller account holder spoke with account holder elisha hv ptm switch query provided information,0,Account,3-818982083047,13.0,NaN,NaN,NaN,Account,14,Y
593,help with ee app online linked customer account will be sent email to verify access,"['help', 'with', 'ee', 'app', 'online', 'linked', 'customer', 'account', 'will', 'be', 'sent', 'email', 'to', 'verify', 'access']",help with ee app online linked customer account will be sent email to verify access,help with ee app online linked customer account will be sent email to verify access,0,Account,3-815565847734,11.0,NaN,NaN,NaN,Account,15,Y
602,customer called having issue creatinine my ee online account talked customer through creating online account,"['customer', 'called', 'having', 'issue', 'creatinine', 'my', 'ee', 'online', 'account', 'talked', 'customer', 'through', 'creating', 'online', 'account']",customer called having issue creatinine my ee online account talked customer through creating online account,customer called having issue creatinine my ee online account talked customer through creating online account,0,Account,3-816147898843,12.0,NaN,NaN,NaN,Account,15,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4809,line dropped.......................,"['line', 'dropped']",line dropped,line dropped,12,Not enough data to classify,NaN,2.0,NaN,NaN,NaN,Not Enough data to classify,2,Y
4810,installed - \n\nflashing orange \n\n\nxbox\n\n,"['installed', 'flashing', 'orange', 'xbox']",installed flashing orange xbox,installed flashing orange xbox,12,Not enough data to classify,NaN,5.0,NaN,NaN,NaN,TV,4,Y
5466,acc number confirmed price Ã‚Â£ per month fibre essentials Ã‚Â£ with off,"['acc', 'number', 'confirmed', 'price', 'ã‚â£', 'per', 'month', 'fibre', 'essentials', 'ã‚â£', 'with', 'off']",acc number confirmed price ã‚â£ per month fibre essentials ã‚â£ with off,acc number confirmed price ã‚â£ per month fibre essentials ã‚â£ with off,1,Billing,3-812750552755,12.0,NaN,NaN,NaN,Not Enough data to classify,12,Y
5656,cx wanted to discuss mobile assets on ee mobile acc passed to mobile billing,"['cx', 'wanted', 'to', 'discuss', 'mobile', 'assets', 'on', 'ee', 'mobile', 'acc', 'passed', 'to', 'mobile', 'billing']",cx wanted to discuss mobile assets on ee mobile acc passed to mobile billing,cx wanted to discuss mobile assets on ee mobile acc passed to mobile billing,1,Billing,3-813677226885,11.0,NaN,NaN,NaN,EE Mobile,14,Y


In [8]:
df['Topic_TB'].unique()

array(['Digital Voice/Handset', 'Hybrid Connect', 'Broadband', 'Account',
       'Engineering', 'Financial Operation', 'Equipment', 'TV', 'Order',
       'Not Enough data to classify', 'Broadband including Account',
       'Account including Financial Operation',
       'Broadband including Financial Operation', 'EE Mobile', 'Disc',
       'Unknown'], dtype=object)

In [9]:
df['Topic_TB'].value_counts()

Broadband                                  777
Equipment                                  730
Order                                      649
Digital Voice/Handset                      597
Account                                    517
Engineering                                493
Financial Operation                        472
Hybrid Connect                             445
Not Enough data to classify                353
TV                                         330
Disc                                       256
EE Mobile                                  117
Broadband including Account                 65
Account including Financial Operation       33
Broadband including Financial Operation     24
Unknown                                      4
Name: Topic_TB, dtype: int64

In [58]:
Test_sample = df
file_path = '/home/jupyter/test2.csv'
Test_sample.to_csv(file_path, index=False) 

In [16]:
#Gemini LLM EXP
import vertexai
from google.cloud import aiplatform
from vertexai.preview.generative_models import GenerativeModel
from vertexai import generative_models
model = GenerativeModel("gemini-1.0-pro")

vertexai.init(project="bt-netdetsds-aif-exp-prod", location="europe-west2")
parameters = {
    "max_output_tokens": 200,
    "temperature": 0.7,
    "top_p": 1
}

In [ ]:
from time import sleep
#rate limiting
categories = ['broadband', 'service', 're number', 'billing', 'DV', 'apps', 'calls cut', 'equipment', 'service issues', 'account', 'engineer/appointment', 'broadband Issues', 'channels', 're number', 'Home Tech Experts', 'hybrid connect', 'new ee','cease', 'tv', 'mobile', 'tv', 'disc', 'set up box', 'ee id', 'order query', 'ee shop', 'transfer', 'account']
#categories = ['broadband', 'engineering', 'account']
classification = []
for i in range(len(df)):
        description = df.iloc[i]['clean']
        response = model.generate_content(f"Considering the following categories {','.join(categories)}, the call comments '{description}' falls into which category? Please provide ONLY ONE of following categories: {','.join(categories)}, in lowercase.")
        sleep(10)
        #response_text = response.text.lower()
        response_text = response.text.lower()
        if response_text.strip() in categories:
            classification.append(response_text)
            print(f"'{response_text}'")
        else:
            classification.append('unknown')
            print('Unknown')
#https://github.com/GoogleCloudPlatform/generative-ai/issues/344

In [ ]:
pd.set_option('display.max_colwidth', None)
new = list(map(str.strip,classification))
df['Gemini_Topics'] = new

In [ ]:
#SQL Query!
"""
SELECT row_id, asset_id, caller_type_cd, caller_name, comments_long, source_create_date, bq_create_dt, customer_id, sra_resolution_cd, todo_cd FROM `bt-bvp-dataprod-dp-prod.cons_dp_bvp_ov_data_product_rw.bvp_dp_113_bq_ov_cust_interaction_raw` 
WHERE (comments_long IS NOT NULL) and (date(source_create_date) = '2024-03-02') and (comments_long != 'This is a Synthmon test') and (comments_long NOT LIKE '%Robotic%') and ( comments_long NOT LIKE '%We have attempted to robotically%' ) and (comments_long NOT LIKE '%L2C%') and (comments_long NOT LIKE '%This is a CSOC%') and (comments_long NOT LIKE '%SR Number%') and (comments_long NOT LIKE '%T2RComm_Clear%') and (comments_long NOT LIKE '%We have attempted to robotically%') and (comments_long NOT LIKE '%ROBOTIC PROCESS AUTOMATION%') and (comments_long NOT LIKE '%Order Number%') and (todo_cd = 'Inbound Call' or todo_cd = 'Chat' or todo_cd = 'InboundCall' or todo_cd = 'Outbound Call' or todo_cd = 'OutboundCall' ) 
order by source_create_date desc """